In [1]:
from restrictions import *

In [2]:
# Setup categories
FoodCategory.reset()
FoodCategory.define("ANIMAL_PRODUCTS")
FoodCategory.define("MEAT", {"ANIMAL_PRODUCTS"})
FoodCategory.define("DAIRY", {"ANIMAL_PRODUCTS"})
FoodCategory.define("EGGS", {"ANIMAL_PRODUCTS"})
FoodCategory.define("FISH", {"ANIMAL_PRODUCTS"})

FoodCategory.define("NUTS")
FoodCategory.define("CHEESE", {"DAIRY"})
FoodCategory.define("CHICKEN", {"MEAT"})
FoodCategory.define("BEEF", {"MEAT"})
FoodCategory.define("ALMOND", {"NUTS"})

FoodCategory(ALMOND)

In [ ]:
# Define restrictions
vegan = DietaryRestriction({"ANIMAL_PRODUCTS"})
vegetarian = DietaryRestriction({"MEAT", "FISH"})
pescatarian = DietaryRestriction({"MEAT"})
nut_free = DietaryRestriction({"NUTS"})
dairy_free = DietaryRestriction({"DAIRY"})

tag_registry.register_tag("VEGAN", DietaryRestriction({"ANIMAL_PRODUCTS"}), category="ethical")
tag_registry.register_tag("VEGETARIAN", DietaryRestriction({"MEAT", "FISH", "SHELLFISH"}), category="ethical")
tag_registry.register_tag("PESCATARIAN", DietaryRestriction({"MEAT"}), category="ethical")
tag_registry.register_tag("NUT-FREE", DietaryRestriction({"NUTS"}), category="allergen")
tag_registry.register_tag("DAIRY-FREE", DietaryRestriction({"DAIRY"}), category="allergen")

In [4]:
# Create some ingredients
salmon = Ingredient("Salmon Fillet", FoodCategory.get("FISH"), 200)
cheese = Ingredient("Cheddar Cheese", FoodCategory.get("CHEESE"), 113, {"milk"})
chicken = Ingredient("Grilled Chicken", FoodCategory.get("CHICKEN"), 165)
almonds = Ingredient("Sliced Almonds", FoodCategory.get("ALMOND"), 170, {"nuts"})

In [5]:
# People
people = [
    Person("Alex", tag="vegan"),
    Person("Jamie", restriction=DietaryRestriction({"MEAT"})),
    Person("Sam", restriction=nut_free),
]

In [6]:
# Meals
meals = [
    Meal("Grilled Salmon", [salmon]),
    Meal("Chicken Alfredo", [chicken, cheese]),
    Meal("Vegan Bowl", []),
    Meal("Nutty Salad", [almonds]),
    Meal("Plain Rice", []),
]

analyzer = MealCompatibilityAnalyzer(meals, people)

# Print the raw matrix
analyzer.print_matrix()

# Score meals
print("\n=== Meal Compatibility Scores ===")
print(analyzer.score_meals())

# Show top 2 most compatible meals
print("\n=== Most Compatible Meals ===")
print(analyzer.get_most_compatible_meals(top_n=2))

# Universally compatible
print("\n=== Universally Compatible Meals ===")
print(analyzer.get_universally_compatible_meals())



                Alex [VEGAN] Jamie [PESCATARIAN] Sam [NUT-FREE]
Grilled Salmon             ❌                   ✅              ✅
Chicken Alfredo            ❌                   ❌              ✅
Vegan Bowl                 ✅                   ✅              ✅
Nutty Salad                ✅                   ✅              ❌
Plain Rice                 ✅                   ✅              ✅

=== Meal Compatibility Scores ===
Grilled Salmon     2
Chicken Alfredo    1
Vegan Bowl         3
Nutty Salad        2
Plain Rice         3
dtype: int64

=== Most Compatible Meals ===
            Alex [VEGAN]  Jamie [PESCATARIAN]  Sam [NUT-FREE]  \
Vegan Bowl          True                 True            True   
Plain Rice          True                 True            True   

            Compatible_Count  
Vegan Bowl                 3  
Plain Rice                 3  

=== Universally Compatible Meals ===
            Alex [VEGAN]  Jamie [PESCATARIAN]  Sam [NUT-FREE]
Vegan Bowl          True                 Tr

In [7]:
for tag in tag_registry.all_tags():
    print(f"{tag}: excludes {tag_registry.get_tag(tag).excluded}")


VEGAN: excludes {'ANIMAL_PRODUCTS'}
VEGETARIAN: excludes {'FISH', 'SHELLFISH', 'MEAT'}
PESCATARIAN: excludes {'MEAT'}
NUT-FREE: excludes {'NUTS'}


In [8]:
# Custom restriction: VEG + DAIRY
r = DietaryRestriction({"MEAT", "FISH", "SHELLFISH", "DAIRY"})
print(tag_registry.generate_tags(r))  # → ['VEGETARIAN']

['VEGETARIAN', 'DAIRY-FREE']


In [9]:
# Tag setup
tag_registry = TagRegistry()
tag_registry.register_tag("VEGAN", DietaryRestriction({"ANIMAL_PRODUCTS"}))
tag_registry.register_tag("VEGETARIAN", DietaryRestriction({"MEAT", "FISH", "SHELLFISH"}))
tag_registry.register_tag("PESCATARIAN", DietaryRestriction({"MEAT"}))
tag_registry.register_tag("NUT-FREE", DietaryRestriction({"NUTS"}))

# Define person with custom restriction
combo_restriction = DietaryRestriction({"MEAT", "FISH", "SHELLFISH", "NUTS"})
person = Person("Sam", restriction=combo_restriction)

print(person.label())  # Sam [VEGETARIAN | NUT-FREE]

Sam [VEGETARIAN | NUT-FREE]


In [ ]:
print(tag_registry.get_tags_by_category("ethical"))     # ['VEGAN', 'VEGETARIAN', 'PESCATARIAN']
print(tag_registry.get_tags_by_category("allergen"))    # ['NUT-FREE', 'DAIRY-FREE']